In [ ]:
from run_rag import RAGApp

a = RAGApp("datasets\manual.pdf")
a.set_rag_model()
response = a.ask_question("박사 과정의 월급은?")
print(response)

In [ ]:
!pip install langchain_huggingface

In [1]:
import sys
import requests
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel, 
    QFileDialog, QLineEdit, QTextEdit, QFrame
)
from PyQt6.QtCore import Qt

class DragDropLabel(QLabel):
    """드래그 앤 드롭을 지원하는 커스텀 라벨 위젯"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.setText("여기에 PDF 파일을 드래그 앤 드롭 하거나\n아래 버튼을 클릭하세요.")
        self.setStyleSheet("""
            QLabel {
                border: 2px dashed #aaa;
                padding: 20px;
                font-size: 14px;
                background-color: #f9f9f9;
            }
        """)

    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        for url in event.mimeData().urls():
            file_path = url.toLocalFile()
            if file_path.lower().endswith('.pdf'):
                print(f"드롭된 파일: {file_path}")
                self.parent().upload_file(file_path)
                return
        print("PDF 파일이 아닙니다.")

class RAGClientApp(QWidget):
    def __init__(self):
        super().__init__()
        # Flask 서버 주소
        self.upload_url = "http://127.0.0.1:5000/upload"
        self.ask_url = "http://127.0.0.1:5000/ask"
        self.init_ui()

    def init_ui(self):
        """애플리케이션의 전체 UI를 설정합니다."""
        main_layout = QVBoxLayout()

        # --- 1. 파일 업로드 섹션 ---
        upload_group_label = QLabel("1. PDF 파일 업로드")
        upload_group_label.setStyleSheet("font-size: 16px; font-weight: bold; margin-top: 10px;")
        main_layout.addWidget(upload_group_label)
        
        self.drop_label = DragDropLabel(self)
        main_layout.addWidget(self.drop_label)

        self.btn_select = QPushButton("컴퓨터에서 PDF 파일 선택")
        self.btn_select.clicked.connect(self.open_file_dialog)
        main_layout.addWidget(self.btn_select)
        
        self.status_label = QLabel("상태: 대기 중")
        self.status_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        main_layout.addWidget(self.status_label)
        
        # --- 구분선 ---
        separator = QFrame()
        separator.setFrameShape(QFrame.Shape.HLine)
        separator.setFrameShadow(QFrame.Shadow.Sunken)
        main_layout.addWidget(separator)

        # --- 2. 질문/답변 섹션 ---
        qa_group_label = QLabel("2. 질문하기")
        qa_group_label.setStyleSheet("font-size: 16px; font-weight: bold; margin-top: 10px;")
        main_layout.addWidget(qa_group_label)

        # 질문 입력 레이아웃
        question_layout = QHBoxLayout()
        self.question_input = QLineEdit()
        self.question_input.setPlaceholderText("업로드된 PDF에 대해 질문을 입력하세요...")
        question_layout.addWidget(self.question_input)

        self.btn_ask = QPushButton("질문하기")
        self.btn_ask.clicked.connect(self.ask_question)
        question_layout.addWidget(self.btn_ask)
        main_layout.addLayout(question_layout)

        # 답변 출력창
        answer_label = QLabel("답변:")
        main_layout.addWidget(answer_label)
        
        self.answer_output = QTextEdit()
        self.answer_output.setReadOnly(True) # 답변은 수정할 수 없도록 설정
        self.answer_output.setStyleSheet("background-color: #f0f0f0;")
        main_layout.addWidget(self.answer_output)
        
        # 초기에는 질문 섹션을 비활성화
        self.set_qa_section_enabled(False)

        self.setLayout(main_layout)
        self.setWindowTitle("PDF 기반 RAG 질문 답변 시스템")
        self.setGeometry(300, 300, 500, 600)

    def set_qa_section_enabled(self, enabled):
        """질문/답변 섹션의 활성화 상태를 설정합니다."""
        self.question_input.setEnabled(enabled)
        self.btn_ask.setEnabled(enabled)
        self.answer_output.setEnabled(enabled)
        if not enabled:
            self.answer_output.setPlaceholderText("PDF 파일을 먼저 업로드해주세요.")
        else:
            self.answer_output.setPlaceholderText("")


    def open_file_dialog(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "PDF 파일 선택", "", "PDF Files (*.pdf)")
        if file_path:
            self.upload_file(file_path)

    def upload_file(self, file_path):
        filename = file_path.split('/')[-1]
        self.status_label.setText(f"상태: '{filename}' 업로드 및 처리 중... (파일 크기에 따라 시간이 걸릴 수 있습니다)")
        self.set_qa_section_enabled(False) # 처리 중 다시 비활성화
        QApplication.processEvents() # UI가 멈추지 않도록 이벤트 처리

        try:
            with open(file_path, 'rb') as f:
                files = {'file': (filename, f, 'application/pdf')}
                # RAG 처리는 오래 걸릴 수 있으므로 timeout을 넉넉하게 설정
                response = requests.post(self.upload_url, files=files, timeout=300)
            
            response.raise_for_status()
            
            response_data = response.json()
            self.status_label.setText(f"상태: {response_data.get('message', '업로드 및 처리 완료!')}")
            self.set_qa_section_enabled(True) # 처리가 끝나면 질문 섹션 활성화

        except requests.exceptions.RequestException as e:
            self.status_label.setText(f"상태: 업로드 실패 - {e}")
        except Exception as e:
            self.status_label.setText(f"상태: 오류 발생 - {e}")

    def ask_question(self):
        question = self.question_input.text().strip()
        if not question:
            self.answer_output.setText("질문을 입력해주세요.")
            return

        self.answer_output.setText("답변을 생성 중입니다...")
        QApplication.processEvents()

        try:
            payload = {"question": question}
            response = requests.post(self.ask_url, json=payload, timeout=60)
            response.raise_for_status()

            response_data = response.json()
            self.answer_output.setText(response_data.get("answer", "답변을 가져올 수 없습니다."))

        except requests.exceptions.RequestException as e:
            self.answer_output.setText(f"오류: 서버와 통신할 수 없습니다.\n{e}")
        except Exception as e:
            self.answer_output.setText(f"알 수 없는 오류가 발생했습니다.\n{e}")


if __name__ == '__main__':
    try:
        import requests
    except ImportError:
        print("requests 라이브러리가 필요합니다. 'pip install -r requirements.txt' 명령어로 설치해주세요.")
        sys.exit(1)
        
    app = QApplication(sys.argv)
    ex = RAGClientApp()
    ex.show()
    sys.exit(app.exec())



SystemExit: 0

c:\Users\minja\anaconda3\envs\rag\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
